In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/KAN/Repo


/content/drive/MyDrive/KAN/Repo


In [ ]:
!pip install -e .

Obtaining file:///content/drive/MyDrive/KAN/Repo
  Preparing metadata (setup.py) ... done
  Running setup.py develop for pykan


In [ ]:
from kan import KAN
import matplotlib.pyplot as plt
import torch
import numpy as np
import pandas as pd

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [ ]:
# read train and test data
train = pd.read_csv('/content/drive/MyDrive/KAN/mnist_train.csv')
test = pd.read_csv('/content/drive/MyDrive/KAN/mnist_test.csv')

In [ ]:
from torchvision import transforms
# convert data to numpy arrays
x_train = train.drop('label', axis=1).values
x_test = test.drop('label', axis=1).values
y_train = train['label'].values
y_test = test['label'].values

# set target size to scale images
target_size = 8

# define transformation
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((target_size, target_size)),
    transforms.ToTensor()
])

# transform the data
def transform_images(images):
    transformed_images = []
    for image in images:
        image = image.reshape(28, 28).astype(np.uint8)
        image = transform(image)
        image = image.view(-1)
        transformed_images.append(image)
    return torch.stack(transformed_images)

train_input = transform_images(x_train)
test_input = transform_images(x_test)
train_label = torch.tensor(y_train, dtype=torch.int64)
test_label = torch.tensor(y_test, dtype=torch.int64)

In [ ]:
"""# Normalize the datasets
train_normalized = train.copy()
test_normalized = test.copy()

# Normalize all columns except the label column
label_column = 'label'  # Adjust this if your label column has a different name

for column in train_normalized.columns:
    if column != label_column:
        train_normalized[column] = (train_normalized[column]) / 255

for column in test_normalized.columns:
    if column != label_column:
        test_normalized[column] = (test_normalized[column]) / 255"""

"# Normalize the datasets\ntrain_normalized = train.copy()\ntest_normalized = test.copy()\n\n# Normalize all columns except the label column\nlabel_column = 'label'  # Adjust this if your label column has a different name\n\nfor column in train_normalized.columns:\n    if column != label_column:\n        train_normalized[column] = (train_normalized[column]) / 255\n\nfor column in test_normalized.columns:\n    if column != label_column:\n        test_normalized[column] = (test_normalized[column]) / 255"

In [ ]:
"""train_normalized.fillna(train.select_dtypes(include=[np.number]).mean(), inplace=True)
test_normalized.fillna(test.select_dtypes(include=[np.number]).mean(), inplace=True)"""

'train_normalized.fillna(train.select_dtypes(include=[np.number]).mean(), inplace=True)\ntest_normalized.fillna(test.select_dtypes(include=[np.number]).mean(), inplace=True)'

In [ ]:
"""# Ändere alle Einträge von 0 zu 0.1
train_normalized.replace(0, 0.1, inplace=True)
test_normalized.replace(0, 0.1, inplace=True)"""

'# Ändere alle Einträge von 0 zu 0.1\ntrain_normalized.replace(0, 0.1, inplace=True)\ntest_normalized.replace(0, 0.1, inplace=True)'

In [ ]:
"""train= train_normalized.iloc[:10, :65]
test= test_normalized.iloc[:10, :65]
train.value_counts()"""

'train= train_normalized.iloc[:10, :65]\ntest= test_normalized.iloc[:10, :65]\ntrain.value_counts()'

In [ ]:
"""# convert data to tensors
train_input = torch.tensor(train.drop('label', axis=1).values, dtype=torch.float32)
test_input = torch.tensor(test.drop('label', axis=1).values, dtype=torch.float32)
train_label = torch.tensor(train['label'].values, dtype=torch.float32)
test_label = torch.tensor(test['label'].values, dtype=torch.float32)"""

"# convert data to tensors\ntrain_input = torch.tensor(train.drop('label', axis=1).values, dtype=torch.float32)\ntest_input = torch.tensor(test.drop('label', axis=1).values, dtype=torch.float32)\ntrain_label = torch.tensor(train['label'].values, dtype=torch.float32)\ntest_label = torch.tensor(test['label'].values, dtype=torch.float32)"

In [ ]:
train_input.shape, test_input.shape, train_label.shape, test_label.shape

(torch.Size([60000, 64]),
 torch.Size([10000, 64]),
 torch.Size([60000]),
 torch.Size([10000]))

In [ ]:
dataset = {
    'train_input': train_input,
    'train_label': train_label,
    'test_input': test_input,
    'test_label': test_label
}

In [ ]:
dataset['train_input'] = train_input.to(device)
dataset['test_input'] = test_input.to(device)
dataset['train_label'] = train_label.to(device)
dataset['test_label'] = test_label.to(device)

In [ ]:
# Convert labels to Long tensors
dataset['train_label'] = dataset['train_label'].long()
dataset['test_label'] = dataset['test_label'].long()

In [ ]:
input_dim = train_input.shape[ 1 ]   # Anzahl der Eingabefunktionen
output_dim = 10   # Anzahl der Klassen
model = KAN(width=[input_dim, 5, output_dim], grid= 5 , k= 5 , seed= 0)  # Vereinfachte Modellkonfiguration

x_eval contains NaN: False
Result solution shape: torch.Size([64, 5, 10, 1])
x_eval contains NaN: False
Result solution shape: torch.Size([5, 10, 10, 1])
Directory already exists: ./model


In [ ]:
def train_acc():
  return torch.mean((torch.argmax(model(dataset['train_input']), dim=1) == dataset['train_label']).float())

def test_acc():
    return torch.mean((torch.argmax(model(dataset['test_input']), dim=1) == dataset['test_label']).float())


results = model.fit(dataset, opt="LBFGS", steps=100, metrics=(train_acc, test_acc), loss_fn=torch.nn.CrossEntropyLoss(),update_grid=False)

train loss: 7.03e-01 | test loss: 7.02e-01 | reg: 9.18e+01 :  13%|▏| 13/100 [21:14<2:22:11, 98.06s/i


KeyboardInterrupt: 

In [ ]:
train_acc()

In [ ]:
test_acc()

In [ ]:
model.saveckpt('/content/drive/MyDrive/KAN/model_mnist')

In [ ]:
#Ka warum das nicjt geht, ist eigentlich eine Vorhandene Funktion im Repo ^^
#loadckpt('/content/drive/MyDrive/KAN/model_mnist')

In [ ]:
model(dataset['test_input'])

In [ ]:
model.plot(beta=3, scale=5)